In [149]:
#Mocap Arena Camera Calibrate Program
#Mateus Pincho - calibrate intrisics using OpenCV
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [150]:
#Reconstrução do tabuleiro
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize # Square size não interfere na calibração -> somente um fator de escala
                       # o square size não é utilizado na calibração dos cara de stanford
    return X

#Detectando os corners
def detectCorners(images, boardPoints, patternSize):
    worldPoints = []
    imagePoints = [] 

    img_size = 0
    counter = 0
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray.shape[::-1]
        ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
        if ret == True:
            print("Corners found in image " + str(fname)) #- see if corners are found 
            #corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            worldPoints.append(boardPoints)
            imagePoints.append(corners)
            counter+=1

    print("Corners found in " + str(counter) + " images")
    #print(img_size)
    return worldPoints, imagePoints, img_size

#Descobrindo os intrisicos, extrinsecos e o erro de reprojeção 
def calibrate(images, useFisheye, patternSize, squareSize):

    boardPoints = construct3DPoints(patternSize, squareSize)

    worldPoints, imagePoints, imgSize = detectCorners(images, boardPoints, patternSize)

    if useFisheye:
        flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
        calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)

        ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), imagePoints, imgSize, None, None,
                                                                    flags=flagsCalib,criteria=calibrateCriteria)
    else:
        flagsCalib = cv2.CALIB_RATIONAL_MODEL

        ret, cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, None, None,
                                                                flags=flagsCalib)

    print("RMS re-projection error:", ret)
    print("The median re-projection error", np.median(perViewError))
    print("Camera Matrix:\n", cameraMatrix)
    print("Distortion Parameters:\n", k)

    return cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError

In [151]:
dataset    =  ['../../images/virtual/set_5/*.jpg',  # dataset[0]
              '../../images/virtual/set_10/*.jpg',  # dataset[1]
              '../../images/virtual/set_15/*.jpg',  # dataset[2]
              '../../images/virtual/set_20/*.jpg',  # dataset[3]
              '../../images/virtual/set_25/*.jpg',  # dataset[4]
              '../../images/chess/opencv/*.jpg']    # dataset[5]

In [227]:
# escolha o conjunto de imagens
image_set = glob(dataset[0])

# defina os parâmetros da calibração
pattern_size = (7,7)
square_size = 30
useFisheye = False

In [228]:
# # formar os conjuntos de dataset para analisar
# import random

# num_5 = random.sample(range(1,21),5)

# image_set_5 = [image_set[i] for i in num_5]

In [229]:
# check the image_set
print(image_set_5)

['../../images/chess/opencv/left06.jpg', '../../images/chess/opencv/right08.jpg', '../../images/chess/opencv/right02.jpg', '../../images/chess/opencv/left14.jpg', '../../images/chess/opencv/left05.jpg']


In [230]:
camera_matrix, distortion, rotation, translation, _, _, _ = calibrate(image_set, useFisheye, pattern_size, square_size)

Corners found in image ../../images/virtual/set_5/image4.jpg
Corners found in image ../../images/virtual/set_5/image1.jpg
Corners found in image ../../images/virtual/set_5/image2.jpg


Corners found in image ../../images/virtual/set_5/image3.jpg
Corners found in image ../../images/virtual/set_5/image5.jpg
Corners found in 5 images
RMS re-projection error: 0.23174696824654417
The median re-projection error 0.22990884012875903
Camera Matrix:
 [[422.14661882   0.         238.57909834]
 [  0.         421.805195   239.85147633]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[ 8.57694379e+01 -3.63430067e+02  3.73762212e-05  4.32801116e-04
  -8.65594325e+02  8.59439982e+01 -3.64239568e+02 -8.66586078e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [231]:
# Mean Error

rvecs = []
tvecs = []
errors = []

board_points = construct3DPoints(pattern_size, square_size)
board_points = board_points.astype('float32')

for image in image_set:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, detected_image_points = cv2.findChessboardCornersSB(gray, pattern_size, None)

    if ret == True:
        #detected_image_points = cv2.cornerSubPix(gray, detected_image_points, (11,11), (-1,-1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

        _, rvec, tvec = cv2.solvePnP(board_points, detected_image_points, camera_matrix, 0)
        rvecs.append(rvec)
        tvecs.append(tvecs)

        projected_image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, 0)

        error = cv2.norm(detected_image_points, projected_image_points, normType= cv2.NORM_L2) / len(projected_image_points)

        
        errors.append(error)

        print(f"Mean Error of Image {image}:", error, "px")

print("Median error: ", np.median(errors))

Mean Error of Image ../../images/virtual/set_5/image4.jpg: 0.034592071090129774 px
Mean Error of Image ../../images/virtual/set_5/image1.jpg: 0.02987550610222366 px
Mean Error of Image ../../images/virtual/set_5/image2.jpg: 0.036186184167423995 px
Mean Error of Image ../../images/virtual/set_5/image3.jpg: 0.032382637287874536 px
Mean Error of Image ../../images/virtual/set_5/image5.jpg: 0.03276358300447476 px
Median error:  0.03276358300447476


In [232]:
# erro RMS
from math import sqrt
rvecs = []
tvecs = []
errors = []

board_points = construct3DPoints(pattern_size, square_size)

for image in image_set:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, pattern_size, None)

    if ret == True:

        _, rvec, tvec = cv2.solvePnP(board_points, corners, camera_matrix, 0)
        rvecs.append(rvec)
        tvecs.append(tvecs)

        image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, 0)

        error = cv2.norm(corners, image_points, normType= cv2.NORM_L2) / sqrt(len(image_points))
        errors.append(error)

        print(f"RMS Error of Image {image}:", error, "px")

print("Median RMS error: ", np.median(errors))

RMS Error of Image ../../images/virtual/set_5/image4.jpg: 0.2421444976309084 px
RMS Error of Image ../../images/virtual/set_5/image1.jpg: 0.2091285427155656 px
RMS Error of Image ../../images/virtual/set_5/image2.jpg: 0.2533032891719679 px
RMS Error of Image ../../images/virtual/set_5/image3.jpg: 0.22667846101512176 px
RMS Error of Image ../../images/virtual/set_5/image5.jpg: 0.2293450810313233 px
Median RMS error:  0.2293450810313233


In [233]:
error = corners - image_points

error = error.reshape(pattern_size[0]*pattern_size[1],2)

# error_x = error[:,0]
# error_y = 

np.sum(np.absolute(error)) / len(error) 

0.2656958443777902

In [234]:
sqrt(sum((src1(:) - src2(:)).^2))

SyntaxError: invalid syntax (1126110346.py, line 1)